In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#import glob <br>
We can use glob to search for a specific file pattern, or perhaps more usefully, search for files where the filename matches a certain pattern by using wildcard characters.

#impoert cv2 <br>
OpenCV-Python is a library of Python bindings designed to solve computer vision problems.
cv2.imread() method loads an image from the specified file. If the image cannot be read (because of missing file, improper permissions, unsupported or invalid format) then this method returns an empty matrix.

#keras <br>
Keras is a neural network Application Programming Interface (API) for Python that is tightly integrated with TensorFlow, which is used to build machine learning models. Keras’ models offer a simple, user-friendly way to define a neural network, which will then be built for you by TensorFlow. 

#Import OS 
The OS module in Python provides functions for interacting with the operating system. OS comes under Python's standard utility modules. This module provides a portable way of using operating system-dependent functionality. The *os* and *os. path* modules include many functions to interact with the file system.

# Seaborn Library
Seaborn is a library for making statistical graphics in Python. It builds on top of matplotlib and integrates closely with pandas data structures. Seaborn helps you explore and understand your data.

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import glob 
import cv2 
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
import seaborn as sns
from keras.applications.vgg16 import VGG16

from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

#Capture training data and labels into respective lists
images = []
labels = []

#  /content/drive/MyDrive/Output/Generated_Mel_Spectrogram/mel_spectrogram_from_VCC2016-20220517T150102Z-001/mel_spectrogram_from_VCC2016/*

for directory_path in glob.glob("/content/drive/MyDrive/Final_Dataset_Sample/Canny_Augmented_Images/*"):
    label = directory_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.JPG")):
        
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (256, 256))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        images.append(img)
        labels.append(label)
        
X_train, X_test, Y_train, Y_test = train_test_split(images, labels, train_size = 0.7, random_state = 1, shuffle = True)

# Random_state is used to set the seed for the random generator so that we can ensure that the results that we get can be reproduced. Because of the nature of splitting the data in train and test is randomised you would get different data assigned to the train and test data unless you can control for the random factor.
# For example, imagine you have numbers: 1,2,3,4,5 and you want to select two random numbers. Without using a random_state you would get different things: i.e. first 2,5 then 3,4 and so on.
# When you control the random generator by setting the random state to 0, 1 or another number, everyone would get the same random numbers generated: i.e random_state=1 then the random numbers are 1,3 and everyone who will pick two random numbers using random_state=1 from 1,2,3,4,5 will get the same two numbers as you: 1,3.

# random state:
# Whenever randomization is part of a Scikit-learn algorithm, a random_state parameter may be provided to control the random number generator used. Note that the mere presence of random_state doesn’t mean that randomization is always used, as it may be dependent on another parameter, e.g. shuffle, being set.
# -----> if random_state = None : Calling the function multiple times will produce different results.
# -----> if random_state = Integer : Will produce the same results across different calls

/content/drive/MyDrive/Final_Dataset_Sample/Canny_Augmented_Images/Apple___Apple_scab
/content/drive/MyDrive/Final_Dataset_Sample/Canny_Augmented_Images/Apple___Black_rot
/content/drive/MyDrive/Final_Dataset_Sample/Canny_Augmented_Images/Apple___Cedar_apple_rust
/content/drive/MyDrive/Final_Dataset_Sample/Canny_Augmented_Images/Apple___healthy
/content/drive/MyDrive/Final_Dataset_Sample/Canny_Augmented_Images/Blueberry___healthy
/content/drive/MyDrive/Final_Dataset_Sample/Canny_Augmented_Images/Cherry_(including_sour)___Powdery_mildew
/content/drive/MyDrive/Final_Dataset_Sample/Canny_Augmented_Images/Cherry_(including_sour)___healthy
/content/drive/MyDrive/Final_Dataset_Sample/Canny_Augmented_Images/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot


In [ ]:
#Convert lists to arrays        
train_images = np.array(X_train)
train_labels = np.array(Y_train)

In [ ]:
#Convert lists to arrays                
test_images = np.array(X_test)
test_labels = np.array(Y_test)


In [ ]:
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [ ]:
print(test_labels_encoded)

array([5, 2, 0, ..., 4, 1, 4])

In [ ]:
#Split data into test and train datasets (already split but assigning to meaningful convention)
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded


In [ ]:
x_train.shape

(5863, 256, 256, 3)

In [ ]:
# # Normalize pixel values to between 0 and 1
# x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
#One hot encode y values for neural network. 
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [ ]:
#Load model wothout classifier/fully connected layers
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

In [ ]:
#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in VGG_model.layers:
	layer.trainable = False
    
VGG_model.summary()  #Trainable parameters will be 0

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
ResNet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in VGG_model.layers:
	layer.trainable = False
    
VGG_model.summary()  #Trainable parameters will be 0

94781440/94765736 [==============================] - 0s 0us/step


In [ ]:
#feature_extractor1=ResNet_model.predict(x_train)

In [ ]:
#features1 = feature_extractor1.reshape(feature_extractor1.shape[0], -1)

In [ ]:
#print(features1.shape)

(140, 131072)


In [ ]:
#Now, let us use features from convolutional network for RF
feature_extractor=VGG_model.predict(x_train)

In [ ]:
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

In [ ]:
print(features.shape)

(140, 32768)


In [ ]:
X_for_RF = features #This is our X input to RF

In [ ]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 42)

In [ ]:
# Train the model on training data
RF_model.fit(X_for_RF, y_train) #For sklearn no one hot encoding

RandomForestClassifier(n_estimators=50, random_state=42)

In [ ]:
print(RF_model.score(X_for_RF, y_train))

1.0


In [ ]:
#Send test data through same feature extractor process
X_test_feature = VGG_model.predict(x_test)


In [ ]:
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

In [ ]:
print(X_test_features.shape)

(60, 32768)


In [ ]:
#Now predict using the trained RF model. 
prediction_RF = RF_model.predict(X_test_features)
#Inverse le transform to get original label back. 
prediction_RF = le.inverse_transform(prediction_RF)

In [ ]:
print(prediction_RF.shape)

(60,)


In [ ]:
print(test_labels.shape)

(60,)


In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_RF))

Accuracy =  0.5833333333333334


In [ ]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import xgboost as xgb

In [ ]:
xgb_classifier = xgb.XGBClassifier()

In [ ]:
xgb_classifier.fit(X_for_RF, y_train)

XGBClassifier(objective='multi:softprob')

In [ ]:
predictions = xgb_classifier.predict(X_test_features)
prediction_XG = le.inverse_transform(predictions)

In [ ]:
prediction_XG

array(['/content/drive/MyDrive/Segmented_images/Canny_image/Apple_scab_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Apple_healthy_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Apple_healthy_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Cedar_apple_rust_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Cedar_apple_rust_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Apple_Black_rot_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Apple_scab_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Apple_scab_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Apple_healthy_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Apple_scab_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Apple_Black_rot_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_im

In [ ]:
print("Accuracy of Model::",metrics.accuracy_score(test_labels, prediction_XG))

Accuracy of Model:: 0.6166666666666667


In [ ]:
from sklearn.svm import SVC

In [ ]:
svc = SVC(C=1.0, random_state=1, kernel='linear')

In [ ]:
svc.fit(X_for_RF, y_train)

SVC(kernel='linear', random_state=1)

In [ ]:
predictions = svc.predict(X_test_features)
prediction_SVC = le.inverse_transform(predictions)

In [ ]:
prediction_SVC

array(['/content/drive/MyDrive/Segmented_images/Canny_image/Apple_scab_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Apple_healthy_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Apple_healthy_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Apple_Black_rot_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Apple_Black_rot_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Cedar_apple_rust_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Apple_scab_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Apple_Black_rot_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Apple_healthy_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Cedar_apple_rust_segmented',
       '/content/drive/MyDrive/Segmented_images/Canny_image/Apple_Black_rot_segmented',
       '/content/drive/MyDrive/Segmented_image

In [ ]:
print("Accuracy of Model::",metrics.accuracy_score(test_labels, prediction_SVC))

Accuracy of Model:: 0.8666666666666667
